In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt
from nltk import tokenize as tnk
from functools import reduce

In [2]:
# data = tfds.load('imdb_reviews', with_info=True,
#                                as_supervised=True)

In [3]:
def docs_to_gen(file_object):
    while True:
        data = file_object.readline()[:-2] #remove /n
        if not data:
            break
        yield data
f = open('train_data87.txt')

In [4]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    docs_to_gen(f), target_vocab_size=2**11)

In [5]:
tokenizer_en.encode("I ")

[5]

In [6]:
tokenizer_en.decode([35])

'had '

In [7]:
tokenizer_en.save_to_file("my_tok_test")

In [8]:
tfds.features.text.SubwordTextEncoder.load_from_file("my_tok_test").decode([1])

'newline'

In [92]:
f = open('train_data87.txt')
g = docs_to_gen(f)
next(g)
#next(g)
#next(g)

"So many times have I walked on ruins , the remainings of places that I loved and got used to.. At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body.. After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly . <newline> It 's funny , but I felt as if after gaining what I desired so long , what I have lived for my entire life , only then , when I achieved immortality I started truly aging . <newline> <newline> 5 world wars have passed , and now they feel like a simple sickeness that would pass by every so often , I could no longer evaluate the individual human as a being of its own , the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much more advanced , a mecha

In [75]:
t = "I was feckin ' sloshed , mate . First time I ever was in the Big Lemon , and I 'd found me the best feckin ' pub I could imagine , I tell ya what . So I stumble out when it was closin ' time , musta been 'round 4 o'clock in the morning , and made my way through some alleys to find the quaint little AirBnB place I 'd rented for the week . <newline> <newline> A'course , that 's how many a horror story starts , ainnit ? But it was all fun and games at first . There was this bloke I saw comin ' towards me in the alley , dark as it was with only a few lights from some apartments overhead , where the folk were still awake . At least , I thought it was a bloke , but he looked more like there was n't something right with the whole alleyway . Like it was a painting someone had gone and ripped with a knife for some reason , fecked-up as it all looked , and the cut looked sort of weird and silvery , and wavy like a heat haze comin ' from a welder 's torch . <newline> <newline> Now this thing must 've been a good foot or five taller 'n me , but sloshed as I am , I just take off my hat and give it a good `` how-do-you-do '' , 'cause I 'm a nice fellow and I wan na pass , y'see ? Now this thing up and goes give me a `` mighty fine , how about y'self , sir ! '' I feckin ' quite pissed my britches ! More in surprise 'n anythin ' , really , but I keep my composure and just sorta try and pass 'm as he walks past me with his long , spindly legs as silvery as the rest of 'm , with his feet makin ' noises like he 's wearin ' cowboy boots with those thingies on 'm . But then that moment stupid old me finds I forgot my lighter at the pub , and so I turn and ask spindleboots there if he 's got some fire for me . <newline> <newline> Darn buggerer turns around on the spot faster 'n my eyes can see and says , he says `` sure ! I got your fire here . '' And the feckin ' thing stretches out one of his spindly legs and wraps it around me like a cobra and pulls me into itself . Devil 's pits , that felt like I was bein ' burnt layer of skin by layer . He sure got me with that . Good feckin ' joke , mate . <newline> <newline> Must 've been out like a light for ages , but when I woke up , I find myself here . So tell me , Doc : are time travellers really that sought after as subjects of study ? 'Cause I know you found me in your fancy cyberwebs database thing and I ca n't be three hundred years old as my passport would say if I had one , but I 'd really like to not die , y'know ? What do y'say , wan na go and grab a beer at the pub "

In [73]:
#[[float(y) for y in x] for x in l]

def short_story_to_tokens(tokenizer,text):
    paragraphs=text.split(" <newline> <newline> ")
    x = [[tokenizer_en.encode(sentence) for sentence in tnk.sent_tokenize(p)] for p in paragraphs]
    return x
y=short_story_to_tokens(tokenizer_en,t)


In [98]:
res = []
f = open('train_data87.txt')
g = docs_to_gen(f)
#for xx in g:
#    res.append(short_story_to_tokens(tokenizer_en,xx))
res = [short_story_to_tokens(tokenizer_en,x) for x in g]

In [111]:
def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])

In [112]:

num_stories = len(res)
paragraphs = [len(x) for x in res]
num_paragraphs = reduce(lambda x,y: x+y,paragraphs,0) / num_stories
sentences = [[len(x) for x in p] for p in res]
num_sentences = reduce(lambda x,y: x+y,flatten(sentences),0) / num_paragraphs

In [115]:
p_in_story = []
s_in_p = []
t_in_s = []
for story in res:
    p_in_story.append(len(story))
    for p in story:
        s_in_p.append(len(p))
        for s in p:
            t_in_s.append(len(s))

            

In [128]:
percentile = 0.95
p_in_story_percentile = sorted(p_in_story)[0:int(len(p_in_story)*percentile)][-1]
s_in_p_percentile = sorted(s_in_p)[0:int(len(s_in_p)*percentile)][-1]
t_in_s_percentile = sorted(t_in_s)[0:int(len(t_in_s)*percentile)][-1]
[p_in_story_percentile,s_in_p_percentile,t_in_s_percentile]

[36, 8, 47]